In [156]:
# Initialization

# Load base overlay
from pynq.overlays.base import BaseOverlay

# Import libraries
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import numpy as np
import ipywidgets as ipw

# Initialize the RFSoC overlay
base = BaseOverlay('base.bit')

# Start RF clocks
base.init_rf_clks()

# Channels
ADC_CHANNEL_D = 0
ADC_CHANNEL_C = 1
ADC_CHANNEL_B = 2
ADC_CHANNEL_A = 3

In [157]:
# Function to plot real and imaginary data in time domain
def plot_complex_time(data, n, fs=sample_frequency, 
                      title='Complex Time Plot'):
    plt_re_temp = (go.Scatter(x = np.arange(0, n/fs, 1/fs),
                              y = np.real(data), name='Real'))
    plt_im_temp = (go.Scatter(x = np.arange(0, n/fs, 1/fs),
                              y = np.imag(data), name='Imag'))
    return go.FigureWidget(data = [plt_re_temp, plt_im_temp],
                           layout = {'title': title, 
                                     'xaxis': {
                                         'title': 'Seconds (s)',
                                         'autorange' : True},
                                     'yaxis': {
                                         'title': 'Amplitude (V)'}})

# Functions
# Function to convert integer to binary array
def int_to_binary_array(num, num_bits):
    return np.array(list(format(num, f'0{num_bits}b')), dtype=int)

# Function to convert binary array to integer
def binary_array_to_int(binary_array):
    return int(''.join(map(str, binary_array)), 2)

In [158]:
# View configurable settings of ADCs
# help(base.radio.receiver.channel[0].adc_block.MixerSettings)
# help(base.radio.receiver.channel[0].adc_tile)
# print(base.radio.receiver.channel[ADC_D_channel].adc_block.MixerSettings.keys())

In [159]:
# Set the center frequency and sampling frequency
center_frequency = 1.24e9  # Frequency of incoming signal
sample_frequency = 2500e6  # Should be at least 2x the center frequency
downsample_factor = 2  # Choose how much to decimate by i.e '2' decimates 2x
number_samples = 8208  # Between 16 and 32xxx

# Set mixer frequency to center frequency to demodulate carrier frequency
base.radio.receiver.channel[ADC_CHANNEL_D].adc_block.MixerSettings['Freq'] = 1240 # MHz
base.radio.receiver.channel[ADC_CHANNEL_C].adc_block.MixerSettings['Freq'] = 1240 # MHz
base.radio.receiver.channel[ADC_CHANNEL_B].adc_block.MixerSettings['Freq'] = 1240 # MHz
base.radio.receiver.channel[ADC_CHANNEL_A].adc_block.MixerSettings['Freq'] = 1240 # MHz

In [160]:
# Sample
raw_data = []  # Storage for incoming I and Q data
for i in range(0, len(base.radio.receiver.channel)):
    raw_data.append(base.radio.receiver.channel[i].transfer(number_samples))
print(raw_data[ADC_CHANNEL_D])

[-0.00018311-3.05175781e-04j -0.00012207+3.66210938e-04j
  0.00018311-3.05175781e-05j ... -0.00036621-2.13623047e-04j
 -0.00042725+1.83105469e-04j -0.00042725-2.74658203e-04j]


In [161]:
# Plotting raw data in the time domain
figs = [] # Storage for ADC time domain plots

# Add plots to figs
for i in range(0, len(base.radio.receiver.channel)): # Show all plots
#for i in range(0, ADC_CHANNEL_D+1): # Show only ADC D
    figs.append(plot_complex_time(
        data=raw_data[i], 
        n=number_samples,
        title=''.join(['Time Domain Plot of ADC Channel ', str(i), ' Before Downconversion'])))
    
# plot ADC real and imaginary time domain data
ipw.VBox(figs) # Only show ADC D

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

In [162]:
# Downconvert
downconverted_data = []  # Storage for downconverted data
for i in range(0, len(base.radio.receiver.channel)):
    downconverted_data.append(signal.decimate(raw_data[i], downsample_factor))
number_samples = number_samples / downsample_factor
print(downconverted_data)

[array([-1.77761831e-04-3.06688352e-04j, -5.67366435e-05+1.27182422e-04j,
        1.68553619e-04-2.31830404e-04j, ...,
        1.64149204e-04+1.29892266e-04j, -1.36560299e-04+3.60161527e-04j,
       -3.68629544e-04-9.57737167e-05j]), array([ 0.00066664+6.13035101e-04j, -0.00013478-2.90870051e-04j,
       -0.00019507-6.60018325e-06j, ..., -0.00037817-1.40733097e-04j,
       -0.00022597+3.66497925e-04j, -0.0001445 +2.81466050e-04j]), array([-3.03610452e-04+9.36733496e-05j, -1.70016458e-04-1.92711945e-04j,
        8.11936090e-05-2.07104960e-04j, ...,
        7.46223242e-05+1.06599781e-05j,  3.64751076e-04+1.35148883e-04j,
        2.12059821e-04-5.39277693e-05j]), array([-5.44229394e-04-3.09358554e-04j,  7.79388158e-05-3.95298548e-06j,
        9.99450237e-05-7.18249843e-05j, ...,
        2.20253705e-04-2.65572136e-04j,  2.20214246e-04+2.16421604e-04j,
        1.85996668e-04+3.50217394e-04j])]


In [163]:
# Plotting downconverted data in the time domain
figs = [] # Storage for ADC time domain plots

for i in range(0, len(base.radio.receiver.channel)): # Show all plots
# for i in range(0, ADC_CHANNEL_D+1): # Show only ADC D
    figs.append(plot_complex_time(
        data=downconverted_data[i],
        n=number_samples,
        title=''.join(['Time Domain Plot of ADC Channel ', str(i), ' After Downconversion'])))
    
# plot ADC real and imaginary time domain data
ipw.VBox(figs) # Only show ADC D 

    'data': [{'name': 'Real',
              'type': 'scatter',
              'ui…

In [ ]:
window = np.array(np.blackman(number_samples)[:])
# print("window:", window)
# print("cdata:", cdata[0])

# downsample_window = np.array(np.blackman(number_samples/downsample_factor)[:])
# print("window:", window)
# print("cdata:", cdata[0])

wdata = []
wfigs = []
wdata.append(cdata[0]*window)
# print("wdata:", wdata[0])
# wfigs.append(plot_complex_time(
#         data=wdata[0], 
#         title=''.join(['Windowed Plot of ADC Channel ', str(2), ' Before Downconversion'])))

# wdata.append(downsampled_signal*downsample_window)
# wfigs.append(plot_complex_time(
#         data=wdata[1], 
#         title=''.join(['Windowed Plot of ADC Channel ', str(2), ' After Downconversion'])))

# ipw.VBox(wfigs)

In [ ]:
fdata = []
for i in range(0, len(wdata)):
    fdata.append(np.fft.fftshift(np.fft.fft(wdata[i])))

In [ ]:
def plot_complex_spectrum(data, N=number_samples, fs=sample_frequency, 
                          title='Complex Spectrum Plot', units='dBW', fc=0):
    plt_temp = (go.Scatter(x = np.arange(-fs/2, fs/2, fs/N) + fc,
                           y = data, name='Spectrum'))
    return go.FigureWidget(data = plt_temp,
                           layout = {'title': title, 
                                     'xaxis': {
                                         'title': 'Frequency (Hz)',
                                         'autorange': True},
                                     'yaxis': {
                                         'title': units}})

magfigs = []
for i in range(0, len(fdata)):
    magfigs.append(plot_complex_spectrum(
            data=abs(fdata[i]),
            title=''.join(['Frequency Magnitude Plot of ADC Channel ', str(i)]),
            units='|Y(f)|',
            fc=round(abs(base.radio.receiver.channel[i].adc_block.MixerSettings['Freq']))*1e6))

ipw.VBox(magfigs)